In [1]:
from app.init import init_llm
init_llm(2)

Init AIPROXY env


<module 'openai' from '/Users/lingoace/Documents/GitHub/llm-101/venv/lib/python3.10/site-packages/openai/__init__.py'>

## Parital Prompt Templates
对于常规的提示模版，需要先准备好input_variables 变量才行。

部分提示模版就是可以让某个变量来部分提示模版，然后等待剩余的变量都准备OK。

部分提示模版也可以做参数传递，这样就会比较灵活，而非要全部input_variables 都准备OK了，且都在一个位置才行。

docs: https://python.langchain.com/en/latest/modules/prompts/prompt_templates/examples/partial.html

In [5]:
from langchain.prompts import PromptTemplate

# 部分字符串
prompt = PromptTemplate(
    template="{foo}{bar}", 
    input_variables=['foo', 'bar']
)

# 场景1: 变量都准备
print(prompt.format(foo='foo', bar='bar'))

# 场景2: 变量部分准好

# 在某个时机，foo值准备好了
partial_prompt = prompt.partial(foo='foo')

# 在另外一个时机，bar值也准备好了
print(partial_prompt.format(bar='bar'))

foobar
foobar


In [6]:
# 部分函数：通过通用的方式获取变量注入到prompt中
# 场景：获取当前日期，不能在提示中对其进行硬编码，并且将其与其他输入变量一起传递有点烦人

from datetime import datetime
def _get_now():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

prompt = PromptTemplate(
    template="Tell me a {topic} joke about the day {date}",
    input_variables=['topic', 'date']
)

partial_prompt = prompt.partial(date=_get_now)
print(partial_prompt.format(topic='funny'))

Tell me a funny joke about the day 2023-06-14 13:16:04


In [7]:
# you can also just initialize the prompt with the partial function
prompt = PromptTemplate(
    template="Tell me a {topic} joke about the day {date}",
    input_variables=['topic'],
    partial_variables={'date': _get_now}
)

print(prompt.format(topic='funny'))

Tell me a funny joke about the day 2023-06-14 13:21:42


## Prompt composition 提示组合

docs: https://python.langchain.com/en/latest/modules/prompts/prompt_templates/examples/prompt_composition.html

将多个提示组合一起，当要重复使用部分提示时会很有用。

通过 `PipelinePrompt` 来实现，两部分：

1. `final_prompt`: 最终提示
2. `pipeline_prompts`: 元祖列表

In [8]:
from langchain.prompts.pipeline import PipelinePromptTemplate

# full template, 由 introduction, example, start 三部分组成
full_template = """{introduction}

{example}

{start}"""
full_prompt = PromptTemplate.from_template(full_template)

# 1.introduction

introduction_template = """You are impersonating {person}."""
introduction_prompt = PromptTemplate.from_template(introduction_template)

# 2.example
example_template = """Here's an example of an interaction: 

Q: {example_q}
A: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

# 3.start
start_template = """Now, do this for real!

Q: {input}
A:"""
start_prompt = PromptTemplate.from_template(start_template)

In [10]:
input_prompts = [
    ('introduction', introduction_prompt),
    ('example', example_prompt),
    ('start', start_prompt)
]

pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)

pipeline_prompt.input_variables

['person', 'example_a', 'input', 'example_q']

In [11]:

print(pipeline_prompt.format(
    person="Elon Musk",
    example_q="What's your favorite car?",
    example_a="Telsa",
    input="What's your favorite social media site?"
))

You are impersonating Elon Musk.

Here's an example of an interaction: 

Q: What's your favorite car?
A: Telsa

Now, do this for real!

Q: What's your favorite social media site?
A:
